In [60]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer, util

#here we are formatting the data in the json file to be fit as training data

embedmodel = SentenceTransformer('nq-distilbert-base-v1')

questionsraw = []
tagsraw = []
qtpairsraw = []

with open('intents.json', encoding='utf-8-sig') as file:
    data = json.load(file)
    for qanda in data['intents']:
        tagsraw.append(qanda['tag'])
        for q in qanda['questions']:
            questionsraw.append(q)
            qtpairsraw.append((q, qanda['tag']))
            
tagsraw = sorted(set(tagsonly))

questionsencoded = []
tagsencoded = []

for (q, t) in qtpairsraw:
    question = embedmodel.encode(q)
    questionsencoded.append(question)
    
    tag = tagsraw.index(t)
    tagsencoded.append(tag)
    
questionsencoded = np.array(questionsencoded)
tagsencoded = np.array(tagsencoded)

In [70]:
#creating a dataset accessible with data loader so we can iterate over the training data, since the data is accessible with indexing

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ChatBot(Dataset):
    def __init__(self):
        self.n_samples = len(questionsencoded)
        self.x_data = questionsencoded
        self.y_data = tagsencoded
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

    
class NeuralNet(nn.Module):
    def __init__(self, inputsize, hiddensize, outputsize): #where inputsize is the number of different question patterns, and outputsize is the number of different tags
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(inputsize, hiddensize)
        self.l2 = nn.Linear(hiddensize, hiddensize)
        self.l3 = nn.Linear(hiddensize, outputsize)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out
    
#hyperparameters
dataset = ChatBot()
batchsize = 8 #can change this
inputsize = len(questionsraw) #number of different questions
hiddensize = 8 #can change this
outputsize = len(tagsraw) #number of different tags
learningrate = 0.001 #can change this
epochs = 10 #can change this


loaddata = DataLoader(dataset = dataset, batch_size = batchsize, shuffle=True, num_workers=0)

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(inputsize, hiddensize, outputsize) #.to(device)

In [71]:
#loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learningrate)

for epoch in range(epochs):
    for (questionsencoded, tagsencoded) in loaddata:
        questionsencoded = questionsencoded #.to(device)
        tagsencoded = tagsencoded #.to(device)
        
        #forwarding
        outputs = model(questionsencoded)
        loss = criterion(outputs, tagsencoded)
        
        #optimizing
        optimizer.zero_grad()
        loss.backwards()
        optimizer.step()
        
    #if (epoch + 1) % 100 == 0:
    print(f'epoch {epoch+1}/{epochs}, loss={loss.item():.4f}')
        
print(f'final loss, loss={loss.item():.4f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x768 and 90x8)